In [ ]:
#intall libraries to convert PDF to JPG
!pip install pdf2image
from pdf2image import convert_from_path, convert_from_bytes
!apt-get install poppler-utils 
# library for creating data paths
import os
import shutil #move files
# library for performing numerical computations
import numpy as np

#imageprocessing and classification
import tensorflow as tf 
from tensorflow import keras

# importing all the required sub-modules from keras
from tensorflow.keras.optimizers import Adam  #can use adam_v2 too, from keras.optimizer import adam_v2
from keras.models import Sequential, Model    #Model for Transfer learning
from keras.applications.vgg16 import VGG16    # Transfer learning here too (AGV comments), 
from keras.preprocessing.image import img_to_array, load_img  # to load images into arrays
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from keras.preprocessing.image import ImageDataGenerator
#where the complete pdf scans are
parent_dir = '/content/drive/MyDrive/ML Training/ScannedDocProject/pdfByName/'
# folders to file by form page
filing_dir=os.path.join(parent_dir,'Filed')
Form_A_dir = os.path.join(filing_dir, 'FormA')
Form_B_dir = os.path.join(filing_dir, 'FormB')
Form_C_dir = os.path.join(filing_dir, 'FormC')
Form_D_dir = os.path.join(filing_dir, 'FormD')
Form_E_dir = os.path.join(filing_dir, 'FormE')
pdf_dir=os.path.join(parent_dir,'Docs')
jpg_dir=os.path.join(parent_dir,'jpegpages')
doc_file_names = os.listdir(pdf_dir)
doc_file_paths = [os.path.join(pdf_dir, file_name) for file_name in doc_file_names[:len(doc_file_names)]]
jpg_file_names = os.listdir(jpg_dir)
jpg_file_paths = [os.path.join(jpg_dir, file_name) for file_name in jpg_file_names[:len(jpg_file_names)]] 
os.chdir('/content/drive/MyDrive/ML Training/ScannedDocProject/pdfByName/jpegpages') 

#convert pdfs
images=[]
for i, img_path in enumerate(doc_file_paths):
    images.append(convert_from_path(img_path))
for i in range(len(images)):
  # Save pages as images in the pdf
  img=images[i]
  for j in range(len(img)):
      img[j].save(str(doc_file_names[i]).replace(".pdf", "")+'page'+str(j) +'.jpg', 'JPEG')
#load model
model = VGG16(include_top=True, weights='imagenet')   
transfer_layer = model.get_layer('block5_pool')
vgg_model = Model(inputs=model.input, outputs=transfer_layer.output)
for layer in vgg_model.layers[0:17]:  #We are saying from layer 0 to 17 
    layer.trainable = False
new_model = Sequential() 
new_model.add(vgg_model)
new_model.add(Flatten())
new_model.add(Dense(5, activation='softmax'))
optimizer = Adam(learning_rate=1e-4)
loss = 'categorical_crossentropy'
metrics = ['accuracy']
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
new_model.load_weights('/content/drive/MyDrive/ML Training/ScannedDocProject/VGG16_2.h5') #please check for lastest model


for i, img_path in enumerate(jpg_file_paths):
  image = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
  array = tf.keras.preprocessing.image.img_to_array(image)
  array=np.expand_dims(array, axis=0)
  pred=new_model.predict(array).argmax()
  if pred==0:
    shutil.move(img_path, Form_A_dir)
  if pred==1:
    shutil.move(img_path, Form_B_dir)
  if pred==2:
    shutil.move(img_path, Form_C_dir)
  if pred==3:
    shutil.move(img_path, Form_D_dir)
  if pred==4:
    shutil.move(img_path, Form_E_dir)
print(len(os.listdir(Form_A_dir)))
print(len(os.listdir(Form_B_dir)))
print(len(os.listdir(Form_C_dir)))
print(len(os.listdir(Form_D_dir)))
print(len(os.listdir(Form_E_dir)))


In [ ]:
def empty_folder(folder_dir):
  for f in os.listdir(folder_dir):
    os.remove(os.path.join(folder_dir, f))
    #this function will empty out the files in a directory
    #run this for each filed folder if you need to clear out the form filed folders 